In [1]:
# Parth korat
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\parth\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [3]:
# Visit the URL with San Diego County zip codes
url = 'https://www.zillow.com/browse/homes/ca/san-diego-county/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [22]:
# find 'ul' tag with the class 'bh-body-links' tag that contains all zip codes
ul =soup.find('ul', class_='bh-body-links')

zip_codes = []

# find the 'li' tag and get all children under the tag
zip_code_tags = ul.findChildren('li', recursive=False)

# run a for loop to get all zip codes, then append to zip_codes list
for zip_code_tag in zip_code_tags:
    zip_codes.append(zip_code_tag.text.strip('\n'))

# show first 5 zip codes from the scrape
zip_codes[:5]

['91910', '92101', '92126', '91911', '92592']

In [23]:
# create dictionary with the 'San Diego County Zip Codes' as the key and the zip_codes list as the values
dict_zip_codes = {'San Diego County Zip Codes': zip_codes}

# read the dictionary as a DataFrame object
zip_codes_df = pd.DataFrame(dict_zip_codes)

zip_codes_df.head()

,San Diego County Zip Codes
0,91910
1,92101
2,92126
3,91911
4,92592
